In [6]:
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import torch.nn.functional as F
import librosa
import numpy as np
from torchvision import models

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
def audio_to_spectrogram(audio_path, sr = 22050, n_fft = 2048, hop_length = 512):
    y, sr = librosa.load(audio_path,sr = sr)
    spectrogram = librosa.stft(y, n_fft = n_fft, hop_length = hop_length)
    magnitude = np.abs(spectrogram)
    return torch.tensor(magnitude).to(device), sr

In [5]:
def GramMatrix(input):
    # A = batch
    # B = Number of feature maps
    # C,D = Dimension of feature maps
    a,b,c,d = input.size()

    features = input.view(a*b,c*d)

    G = torch.mm(features, features.t())

    #Normalize by dividing by total
    return G.div(a*b*c*d)

In [8]:
vgg = models.vgg19(pretrained=True).features.to(device)

c:\Users\xkhai\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\xkhai\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\xkhai/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [01:53<00:00, 5.08MB/s] 


In [10]:
for param in vgg.parameters():
    param.requires_grad = False

In [11]:
def extract_features(x, model, layers):
    features = []
    for i, layer in enumerate(model):
        x = layer(x)
        if i in layers:
            features.append(x)
    return features

In [12]:
def style_loss(content_grams, style_grams):
    loss = 0
    for content_gram, style_gram in zip(content_grams, style_grams):
        loss += F.mse_loss(content_gram, style_gram)
    return loss

class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear()
        self.linear2 = nn.Linear()

        self.N = torch.distributions.Normal(0,1)
        self.N.loc = self.N.loc.cuda()
        self.N.scale = self.N.scale.cuda()
        self.kl = 0

    def forward(self,x):
        x = torch.flatten(x,dim)
        x = F.leaky_relu(self.linear1)
        return 

class Decoder(nn.module):
    def __init__(self,latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims,512)
        self.linear2 = nn.Linear(512,1024)
    
    def forward(self,z):
        z = F.leaky_relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return original_shape

class AutoEncoder(nn.Module):
    def __init__(self,latent_dims):
        super(AutoEncoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self,x):
        z = self.encoder(x)
        return self.decoder(z)

def train(autoencoder,data,epoch = 20):
    opt = torch.optim.adamw(autoencoder.parameters())
    for epoch in range(epoch):
        
